In [1]:
import os
import time
import torch
import torchvision.transforms as transforms
import numpy as np
import glob
from PIL import Image

In [2]:
from google.colab import drive
drive.mount("/content/drive")
import sys

filepath = "/content/drive/My Drive/Basics to ai/CoE202_Final_Project_Distribution_package/code_and_data"  # Change this path for your Google drive setting.
sys.path.append(filepath)

from model_baseline import cancer_classifier
# Upload util.py and model_baseline.py on your Google Drive filepath.

Mounted at /content/drive


In [3]:
class cancer_test_dataset(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.files = glob.glob(f"{root}/*.jpg")
    
    def __len__(self):
        ans = len(self.files)
        return ans

    def __getitem__(self, idx):
        image = Image.open(f"{self.root}/{str(idx + 1)}.jpg")
        if self.transform is not None:
            image = self.transform(image)
        return image

In [4]:
experiment = "exp_12Dec, 8 layer"  # Write down experiment name (For saving the result)
state_path = f"{filepath}/{experiment}/model_state_dict.pt"  # Write down your model location

model = cancer_classifier().cuda()
print(model)
model.load_state_dict(torch.load(state_path))
model.eval()

# If loading is done with no error, you will see  " <All keys matched successfully> "

cancer_classifier(
  (conv1): Conv2d(3, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(56, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(256, 640, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(640, 640, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(640, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(512, 420, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(420, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout025): Dropout2d(p=0.25, inplace=False)
  (dropout050): Dropout2d(p=0.5, inplace=False)
  (linear1): Linear(in_features=5120, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=2, bias=True)
)


cancer_classifier(
  (conv1): Conv2d(3, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(56, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(256, 640, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(640, 640, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(640, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(512, 420, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(420, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(320, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (dropout025): Dropout2d(p=0.25, inplace=False)
  (dropout050): Dropout2d(p=0.5, inplace=False)
  (linear1): Linear(in_features=5120, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (linear3): Linear(in_features=1024, out_features=2, bias=True)
)

In [5]:
## Data load
test_dataset = cancer_test_dataset(root=f"{filepath}/test/",
                                                transform=transforms.Compose([
                                                    transforms.ToTensor()
                                                ]))
test_loader = torch.utils.data.DataLoader(test_dataset,
                                        batch_size=4,
                                        shuffle=False,
                                        drop_last=False
                                        )

In [6]:
prediction_index = []
accuracy_list = []

for _, (batch) in enumerate(test_loader):
    batch = batch.cuda()
    prediction = model(batch)
    _, i = prediction.max(dim=1)
    prediction_index += i.tolist()

final_index = prediction_index
print(f"Example prediction, 5 items from front : {final_index[:5]}")
print(f"""{time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())} || Test finished!""")
np.savetxt(f"{filepath}/{experiment}/test_result.csv", np.dstack((np.arange(1, len(final_index)+1), final_index))[0], "%d,%d", header="Id,Values")

Example prediction, 5 items from front : [0, 0, 1, 1, 1]
2020-12-13 02:55:55 || Test finished!
